# Generating a Bi-Monthly Maximum Water Course Level dataset


In [83]:
import pybomwater.bom_water as bm
from sidecar import Sidecar
from ipyleaflet import Map, GeoJSON, Polygon
from IPython.display import display, clear_output, HTML, JSON
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'jupyterlab'
from plotly.subplots import make_subplots
import shapely
import shapely.geometry
import json
import requests
import sys
from geojson import Feature, FeatureCollection, Point
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import datetime as dt
import logging
import sys
from pint import UnitRegistry, set_application_registry
import pint_xarray
from datetime import datetime, timedelta
import xarray as xr
import cftime


### Setup some constants

In [57]:
#setup unit registry for later use.  Using the library of units we can be consistent 
ureg = UnitRegistry(force_ndarray=True)
set_application_registry(ureg) #sync this ureg with xarray unit registry

display(f'The unit for a day: {ureg.d}, used for daily data')

'The unit for a day: day, used for daily data'

In [58]:
#Get an instance of the api package that will be used to access the BoM service

_bm = bm.BomWater()
procedure = _bm.procedures.Pat3_C_B_1_DailyMax 
prop = _bm.properties.Water_Course_Level
display(f'Procedure: {procedure}, Property: {prop}')

'Procedure: Pat3_C_B_1_DailyMax, Property: Water Course Level'

In [60]:
def write_json_features(features, file):
    collection = FeatureCollection(features)
    with open(file, "w") as f:
        f.write('%s' % collection)

In [59]:
logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(filename='tmp5a.log'),
        logging.StreamHandler(sys.stdout)
    ]
)

logger = logging.getLogger('TS_Saver')

### Bounding Box for the MDB

This bounding box is to do a first pass filter 

In [61]:
low_left_lat = -37.505032
low_left_long = 138.00
upper_right_lat = -24.00
upper_right_long = 154.00

lower_left_coords = f'{low_left_lat} {low_left_long}'
upper_right_coords = f'{upper_right_lat} {upper_right_long}'
coords = tuple((lower_left_coords, upper_right_coords))

### Filtering gauges using the bounding box and static station Id list

In [62]:
station_info=[{"station_id": "A4260903", "crest_height": 3.3},
{"station_id": "A4260519", "crest_height": 6.1},
{"station_id": "A4260517", "crest_height": 9.8},
{"station_id": "A4260515", "crest_height": 13.2},
{"station_id": "A4260513", "crest_height": 16.3},
{"station_id": "A4260511", "crest_height": 19.2},
{"station_id": "A4260509", "crest_height": 22.1},
{"station_id": "A4260507", "crest_height": 24.6},
{"station_id": "A4260505", "crest_height": 27.4},
{"station_id": "425010", "crest_height": 30.8},
{"station_id": "414209", "crest_height": 47.9}]
response = _bm.request(_bm.actions.GetFeatureOfInterest, None, prop, procedure, None, None, lower_left_coords, upper_right_coords  )
lock_sites = []
if response.status_code == 200:
    response_json = _bm.xml_to_json(response.text)
    '''bomwater creates a FeatureCollection which can be used for mapping'''
    feature_list = _bm.create_feature_list(response_json, None )
    for f in feature_list['features']:

        # if f.properties['stationNo'] in station_ids:
        found = list(filter(lambda x:x["station_id"]==f.properties['stationNo'],station_info))
        if found:
            print(f'Match found: {f} and {found}')
            lock_sites.append(f)
            
else:
    print(response.status_code)


Match found: {"geometry": {"coordinates": [142.759783, -34.599668], "type": "Point"}, "properties": {"long_name": "http://bom.gov.au/waterdata/services/stations/414209", "name": "MURRAY_@_LOCK_15", "stationId": null, "stationNo": "414209"}, "type": "Feature"} and [{'station_id': '414209', 'crest_height': 47.9}]
Match found: {"geometry": {"coordinates": [141.9045, -34.11], "type": "Point"}, "properties": {"long_name": "http://bom.gov.au/waterdata/services/stations/425010", "name": "MURRAY@L10_WENTWORTH", "stationId": null, "stationNo": "425010"}, "type": "Feature"} and [{'station_id': '425010', 'crest_height': 30.8}]
Match found: {"geometry": {"coordinates": [139.615749, -34.35095], "type": "Point"}, "properties": {"long_name": "http://bom.gov.au/waterdata/services/stations/A4260903", "name": "River_Murray_at_Lock_1_Downstream_(AMTD_274.3km)", "stationId": null, "stationNo": "A4260903"}, "type": "Feature"} and [{'station_id': 'A4260903', 'crest_height': 3.3}]
Match found: {"geometry": {

### Process and save raw and result data
- Load required gauge data 
- Convert to desired timestep (eg bi monthly max)
- Write all data to disk (raw and processed)

In [82]:
def create_da_daily_dates(start_date, days_since_start):
    datetime_values = xr.DataArray([start_date + timedelta(days=int(days)) for days in days_since_start])
    return datetime_values

def get_date_from_str(contains_string: str):
    # Parse the string to obtain the datetime object
    result_datetime = cftime.datetime.strptime(contains_string, 'days since %Y-%m-%d %H:%M:%S')
    return result_datetime

def hydro_plot_data(da, var_name, title=None, fig=None):
    single_plot = False
    if fig == None:
        fig = go.Figure()
        single_plot = True
    if not hasattr(da.data,'units'):
        raise Exception('This data array has no units!!')
    unit = da.data.units
 
    if not isinstance(da['time'].values[0], np.datetime64):
        start_date = get_date_from_str(da['time'].attrs.get('units', None))
        dates = create_da_daily_dates(start_date=start_date, days_since_start=da['time'].values)
    else:
        dates = da.time
    fig.add_trace(go.Scatter(x=dates, y=da, mode='lines', name=f'{var_name} {unit}', line_shape='linear'))
    # Customize the layout (optional)
    if title:
        graph_title = f'{title}'
    else:
        graph_title = f'{var_name} Hydrograph'
    fig.update_layout(
        title=graph_title,
        xaxis_title='Time',
        yaxis_title=f'{var_name} ({unit})',
        xaxis_tickformat = '%d %B (%a)<br>%y'
        # xaxis=dict(tickmode='linear'),
    )

    if single_plot:
        fig.show()
    else:
        return fig

In [84]:
'''Note: Folders should exist before running'''
#Create a dataframe to frame a constant period. As gauge observation data will have varing start and end dates
date1 = dt.date(1988, 1, 1)
date2 = dt.date(2021, 12, 31)
dates = {'time':pd.date_range(start=date1, end=date2)}
values = pd.Series(-1, range(len(dates)))
period = pd.DataFrame(dates, )
period['time'] = pd.to_datetime(period['time'])
#Define the period requested from the BoM Water service
t_begin = "1988-01-01T01:00:00+10"
t_end = "2021-12-31T00:00:00+10"
#Define the Procedure and Properties to request from the service
procedure = _bm.procedures.Pat3_C_B_1_DailyMax  
prop = _bm.properties.Water_Course_Level
result_set = []

bbox = [None, None]
features = []
for station in lock_sites:
    station_long_name = station['properties']['long_name']
    features.append(station_long_name) 
water_course_levels = _bm.get_observations(features, prop, procedure, t_begin, t_end, bbox)


for sites in lock_sites:
    try:
        #Request from BoM Water server
        # response = _bm.request(_bm.actions.GetObservation, sites['properties']['stationNo'], prop, procedure, t_begin, t_end)
        # ts = _bm.parse_data(response)
        weir_station_no = sites['properties']['stationNo']
        weir_station_longname = sites['properties']['long_name']
        for wcl_chunk in water_course_levels:
            for wcl in wcl_chunk:
                if wcl == weir_station_longname:
                    wcl_da =  wcl_chunk[wcl]
                    break
        
        if len(wcl_da) < 1:
            logger.warning(f'Data length is zero for station: {weir_station_no}')
        else:
            dir = f'./bomwater_data/mdb_water_course_level/'
            #Save raw data
            comp = dict(zlib=True, complevel=5)
            encoding = {var: comp for var in wcl_da.data_vars}
            wcl_da.to_netcdf(f'{dir}{weir_station_no}.nc', format="NETCDF4", engine="netcdf4")#, encoding=encoding)
            
            
            station_data = wcl_da#.to_dataframe()#ts.reset_index()
            #Get Station from filter list
            found = list(filter(lambda x:x["station_id"]==weir_station_no,station_info))
            
            #Resample raw data with static period to make standard length files and so bi monthly periods start at the same month
            station_data = station_data.sel(time=slice(date1, date2))
            
            #Group by bi monthly max
            result = station_data.resample(time='2MS').max()
            
            #Include the crest height with the final dataset
            # result['Structure Crest Height(m)'] = found[0]['crest_height']
            result = result.rename({'Water Course Level [m]': 'BiMonthly Maximum Water_Course_Level(m)'})

            crest_height = xr.DataArray(pd.Series(found[0]['crest_height'], index=result.time), coords=[result.time], dims=['time'])
            result = result.assign(Structure_Crest_Height=lambda x: x.Quality - x.Quality + crest_height )
            result['Structure_Crest_Height'] = result['Structure_Crest_Height'].pint.quantify({'Structure_Crest_Height': ureg.meter})


            #Apply units (meters) to the data variable 
            result['BiMonthly Maximum Water_Course_Level(m)'] = result['BiMonthly Maximum Water_Course_Level(m)'].pint.quantify({'BiMonthly Maximum Water_Course_Level(m)': ureg.meter})

            #Save result to same directory as the raw data
            result.pint.dequantify().to_netcdf(f'{dir}biMonthlyMax{weir_station_no}.nc', format="NETCDF4", engine="netcdf4")

            
            result_set.append(result)
            

            # y = result['BiMonthly Maximum Water_Course_Level(m)']
            # x = result.time
            # plt.figure()
            # plt.title(f'Station ({weir_station_no}) Water Course Levels (m)')
            # plt.ylabel('BiMonthly Maximum Water Course Level (m)')
            # plt.xlabel('Dates')
            # plt.plot( x, y, label=weir_station_no )
            # crest_height = pd.Series(found[0]['crest_height'], index=range(len(result.time)))
            # plt.plot(x, crest_height, label='Crest Height')
            

            # plt.savefig(f'{dir}{weir_station_no}_plot.jpg')
            fig = go.Figure()
            hydro_plot_data( result['BiMonthly Maximum Water_Course_Level(m)'], prop, f'Station ({weir_station_no}) BiMonthly Maximum Water Course Levels (m)', fig)

            hydro_plot_data( result.Structure_Crest_Height, 'Heights', f'Station ({weir_station_no}) BiMonthly Maximum Water Course Levels with Structure Crest Height', fig)
            fig.show()
            
    except:
        e = sys.exc_info()[0]
        weir_station_no = sites['properties']['stationNo']
        logger.warning(f'Processing failed with system info: {e} for station: {weir_station_no}')
        
    

http://bom.gov.au/waterdata/services/stations/414209
http://bom.gov.au/waterdata/services/stations/425010
http://bom.gov.au/waterdata/services/stations/A4260903
http://bom.gov.au/waterdata/services/stations/A4260519
http://bom.gov.au/waterdata/services/stations/A4260517
http://bom.gov.au/waterdata/services/stations/A4260515
http://bom.gov.au/waterdata/services/stations/A4260513
http://bom.gov.au/waterdata/services/stations/A4260511
http://bom.gov.au/waterdata/services/stations/A4260509
http://bom.gov.au/waterdata/services/stations/A4260507
http://bom.gov.au/waterdata/services/stations/A4260505
http://bom.gov.au/waterdata/services/stations/414209
http://bom.gov.au/waterdata/services/stations/425010
http://bom.gov.au/waterdata/services/stations/A4260903
http://bom.gov.au/waterdata/services/stations/A4260519
http://bom.gov.au/waterdata/services/stations/A4260517
http://bom.gov.au/waterdata/services/stations/A4260515
http://bom.gov.au/waterdata/services/stations/A4260513
http://bom.gov.au/

c:\office12\bomwater-notebook\.venv\lib\site-packages\xarray\core\variable.py:305: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



c:\office12\bomwater-notebook\.venv\lib\site-packages\xarray\core\variable.py:305: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



c:\office12\bomwater-notebook\.venv\lib\site-packages\xarray\core\variable.py:305: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



c:\office12\bomwater-notebook\.venv\lib\site-packages\xarray\core\variable.py:305: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



...

c:\office12\bomwater-notebook\.venv\lib\site-packages\xarray\core\variable.py:305: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



c:\office12\bomwater-notebook\.venv\lib\site-packages\xarray\core\variable.py:305: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



c:\office12\bomwater-notebook\.venv\lib\site-packages\xarray\core\variable.py:305: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



c:\office12\bomwater-notebook\.venv\lib\site-packages\xarray\core\variable.py:305: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.



c:\office12\bomwater-notebook\.venv\lib\site-packages\xarray\core\variable.py:305: UnitStrippedWarning:

The unit of the quantity is stripped when downcasting to ndarray.

